Primero, importamos todos los módulos necesarios:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16,9]
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import scale, StandardScaler

A continuación, configuramos matplotlib para este cuaderno:

In [ ]:
% matplotlib inline
plt.rcParams["savefig.dpi"] = 300
plt.rcParams["savefig.bbox"] = "tight"
np.set_printoptions(precision=3, suppress=True)

# DNNs y CNNs con MNIST Permutado

Vamos a ver cómo las Vanilla Neural Networks y las redes convolucionales se comportan de manera distinta ante la pérdida de información bidimensional en una imagen.

En este cuaderno, vamos a cargar MNIST y vamos a alterar las imágenes de forma que para cada imagen, los mismos píxeles permanecen, pero alterados de manera aleatoria en su posición (llamaremos a este dataset MNIST permutado).

De esta forma, veremos que una DNN se comporta igual con MNIST sin permutar y permutado, ya que no está haciendo uso de la información bidimensional que hay en nuestras muestras de entrenamiento. Sin embargo, el comportamiento de la CNN será muy distinto en ambos casos, bajando su rendimiento de manera considerable cuando como resultado de la permutación, se pierde la información bidimensional que cada digito posee.

Importamos el dataset de MNIST directamente desde Keras, y almacenamos las muestras y las etiquetas en sendas variables:

In [ ]:
from keras.datasets import mnist
import keras

(X_train, y_train), (X_test, y_test) = mnist.load_data()

Ahora montamos Google Drive en nuestro entorno para poder hacer operaciones como la carga y guardado de datos:

In [ ]:
#@markdown Decide si guardar los gráficos de drive en una carpeta específica
save_imgs_to_drive = False #@param {type:"boolean"}
if save_imgs_to_drive:
  from google.colab import drive
  drive.mount('/content/drive')

Creamos un directorio donde almacenar las imágenes que vamos a ir generando en el cuaderno:

In [ ]:
if save_imgs_to_drive:
  !mkdir -p "/content/drive/My Drive/Deep Learning/Images"
  images_dir = '/content/drive/My Drive/Deep Learning/Images'

Por último, configuramos algunos parámetros que iremos reutilizando a lo largo del cuaderno:

In [ ]:
#@title Configuracion de parametros
num_classes = 10 #@param {type:"raw"}
batch_size = 128 #@param {type:"raw"}
epochs = 10 #@param {type:"raw"}

# input image dimensions
img_rows, img_cols = 28, 28
input_shape = img_rows * img_cols

Utilizamos MatPlotLib - PyPlot para pintar las cinco primeras figuras del dataset MNIST, para poder compararlas luego con una alteración del dataset que vamos a discutir:

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(12, 3))
for i, ax in enumerate(axes.ravel()):
    ax.imshow(X_train[i, :, :], cmap='gray_r')
    ax.set_xticks(())
    ax.set_yticks(())
if save_imgs_to_drive:
  plt.savefig(f"{images_dir}/mnist_org.png")

Generamos una semilla aleatoria, y a partir de ahí generamos un número de permutaciones igual al número de píxeles que tenemos en las imágenes de MNIST:

In [ ]:
rng = np.random.RandomState(42)
perm = rng.permutation(28 * 28)

In [ ]:
perm

Las operaciones anteriores han generado un array con 784 elementos (28x28), de 0 a 783, ordenados de manera aleatoria. Ya que las dimensiones de nuestro conjunto de muestras de entrenamiento son:

In [ ]:
X_train.shape

, vamos a realizar una permutación por columnas en base al array `perm` que hemos creado antes para alterar el orden de cada uno de los dígitos de MNIST. Una vez hecha esa permutación, volvemos a redimensionar el conjunto de muestras de entrenamiento y de prueba para que sean las que tenemos en los conjuntos sin permutar:

In [ ]:
X_train_perm = X_train.reshape(-1, img_rows * img_cols)[:, perm].reshape(-1, img_rows, img_cols)
X_test_perm = X_test.reshape(-1, img_rows * img_cols)[:, perm].reshape(-1, img_rows, img_cols)

Si volvemos a pintar los cincon primeros dígitos del nuevo dataset de entrenamiento, vemos que las formas ahora no son reconocibles debido a la permutación:

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(12, 3))
for i, ax in enumerate(axes.ravel()):
    ax.imshow(X_train_perm[i, :, :], cmap='gray_r')
    ax.set_xticks(())
    ax.set_yticks(())
if save_imgs_to_drive:
  plt.savefig(f"{images_dir}/mnist_permuted.png")

La diferencia entre una red neuronal convolucional y otras arquitecturas (por ejemplo, una red neuronal densamente conectada) es que la red convolucional no puede aprender nada del dataset permutado, ya que ha desaparecido la estructura bidimensional del mismo, y la información de vecindad entre puntos ha dejado de tener significado. 

Sin embargo, una red neuronal densamente conectada aprendería lo mismo de los dos datasets, ya que el orden de las columnas en los datos no afecta a cómo se aprenden los parámetros.

Veámoslo con más detalle.

### Red neuronal densa

Vamos a crear una red neuronal densa como ya hemos visto con Keras. Utilicemos un modelo secuencial, con una sola capa de 512 neuronas utlizando una función de activación ReLu, y tras ella una capa de clasificación como ya hemos visto:

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential([
    Dense(512, input_shape=(input_shape,), activation='relu'),
    Dense(10, activation='softmax'),
])

Recordemos que vamos a hacer una clasificación categórica en 10 clases diferentes, por lo que tendremos que hacer un *hot encoding* de los vectores de etiquetas transformándolos en matrices binarias. Para ello podemos utilizar la utilidad de Keras `to_categorical`:

In [ ]:
X_train = X_train.reshape(60000, input_shape)
X_test = X_test.reshape(10000, input_shape)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print(f"Dimensiones X_train: {X_train.shape}\n"
      f"Dimensiones X_test: {X_test.shape}")

In [ ]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(f"Dimensiones y_train: {y_train.shape}\n"
      f"Dimensiones y_test: {y_test.shape}")

Compilamos el modelo y mostramos el resumen de lo que acabamos de configurar:

In [ ]:
model.compile("adam", "categorical_crossentropy", metrics=['accuracy'])
model.summary()

Ahora entrenamos, utilizando el callback que el entrenamiento que el modelo devuelve para poder pintar posteriormente la precisión del modelo. En en entrenamiento, vamos a utilizar 10 épocas y un tamaño de lote de 128 muestras, así como reservar un 10% de los datos para validación:

In [ ]:
history_callback_dense = model.fit(X_train, y_train, batch_size=batch_size,
                             epochs=epochs, verbose=1, validation_split=.1)

Por último, salvamos el modelo por si no queremos volver a ejecutar el entrenamiento:

In [ ]:
if save_imgs_to_drive:
  model.save("dense_normal.h5")

### Red Neuronal Convolucional

De la misma forma que hemos entrenado una red neuronal densa convencional, realizamos ahora el entrenamiento de una red neuronal convolucional.


En el caso de una CNN, recordemos que las matrices de entradas tienen que ser tetradimensionales para tener en cuenta los canales de las imágenes:

In [ ]:
X_train_images = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test_images = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten

cnn = Sequential()
cnn.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(Conv2D(32, (3, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(Flatten())
cnn.add(Dense(64, activation='relu'))
cnn.add(Dense(num_classes, activation='softmax'))

Mostramos el aspecto que tiene el modelo así configurado:

In [ ]:
cnn.summary()

In [ ]:
cnn.compile("adam", "categorical_crossentropy", metrics=['accuracy'])
history_cnn = cnn.fit(X_train_images, y_train,
                      batch_size=batch_size, epochs=epochs, verbose=1, validation_split=.1)

### Comparación red densa - CNN simple con información 2D

Definimos una función para filtrar del dataframe las columnas que nos interesan, de cara a que el graficado nos aparezca en las dimensiones correctas:

In [ ]:
def plot_history(logger):
    df = pd.DataFrame(logger.history)
    df[['accuracy', 'val_accuracy']].plot()
    plt.ylabel("accuracy")
    #df[['loss', 'val_loss']].plot(linestyle='--', ax=plt.twinx())
    #plt.ylabel("loss")

In [ ]:
plot_history(history_cnn)

Como puede verse, la precsión en validación y entrenamiento de la red convolucional, aunque no es demasiado sofisticada, supera la de la red densa, como por otro lado era de esperar ya que estamos considerando la información de entorno en la detección de patrones bidimensionales gracias a las convoluciones.

In [ ]:
plot_history(history_callback_dense)

### Comparación red densa - CNN simple con dataset permutado
Recordemos que en este caso, estamos perdiendo la información bidimensional que las imágenes tienen y de la que está haciendo uso la red convolucional. Reconfiguramos la variable `input_shape` ya que en la definición de la red convolucional la alteramos para añadirle una dimensión más:

In [ ]:
input_shape = img_rows * img_cols

Ahora tocamos igual que hicimos anteriormente las dimensiones de las entradas para poder alimentarlas a nuestra red densa:

In [ ]:
X_train_perm = X_train_perm.reshape(60000, input_shape)
X_test = X_test_perm.reshape(10000, input_shape)
X_train_perm = X_train_perm.astype('float32')
X_test_perm = X_test.astype('float32')
X_train_perm /= 255
X_test_perm /= 255
print(f"Dimensiones X_train_perm: {X_train_perm.shape}\n"
      f"Dimensiones X_test_perm: {X_test_perm.shape}")

La red densa ya está definida, vamos a utilizar la misma. En este caso, lo que tenemos que hacer es alimentar la arquitectura con los datos reordenados aleatoriamente para poder analizar posteriormente el comportamiento:

In [ ]:
history_callback_dense_shuffle = model.fit(X_train_perm, y_train, batch_size=batch_size,
                             epochs=epochs, verbose=1, validation_split=.1)

Igualmente, preparamos los NumPy arrays permutados para poder alimentarlos a nuestra red convolucional. Esto exige que añadamos una dimensión más como ya vimos para contabilizar los canales de color de las imágenes:

In [ ]:
X_train_images_perm = X_train_perm.reshape(X_train_perm.shape[0], img_rows, img_cols, 1)
X_test_images_perm = X_test_perm.reshape(X_test_perm.shape[0], img_rows, img_cols, 1)

Una vez hecho esto, compilamos nuestra arquitectura y alimentamos el modelo con los datos permutados, almacenando el resultado en un nuevo callback que luego nos servirá para poder visualizar las precisiones de las diferentes aproximaciones:

In [ ]:
cnn.compile("adam", "categorical_crossentropy", metrics=['accuracy'])

X_train_images_perm = X_train_perm.reshape(X_train_perm.shape[0], img_rows, img_cols, 1) / 255
history_cnn_perm = cnn.fit(X_train_images_perm, y_train,
                           batch_size=128, epochs=10, verbose=1, validation_split=.1)

Ahora llega el momento de visualizar la comparación. Comenzamos con los datos sin permutar:

In [ ]:
cnn = pd.DataFrame(history_cnn.history)
dense = pd.DataFrame(history_callback_dense.history)
dense_perm = pd.DataFrame(history_callback_dense_shuffle.history)
cnn_perm = pd.DataFrame(history_cnn_perm.history)

In [ ]:
res_org = pd.DataFrame({'cnn_train': cnn.accuracy, 'cnn_val': cnn.val_accuracy, 'dense_train': dense.accuracy, 'dense_val': dense.val_accuracy})
res_org.plot()
plt.ylim(.7, 1)
if save_imgs_to_drive:
  plt.savefig("images/mnist_org_curve.png")

Para los datos permutados, las gráficas de percisión para las dos arquitecturas quedan como sigue:

In [ ]:
res_perm = pd.DataFrame({'cnn_train': cnn_perm.acc, 'cnn_val': cnn_perm.val_acc, 'dense_train': dense_perm.acc, 'dense_val': dense_perm.val_acc})
res_perm.plot()
plt.ylim(.7, 1)

if save_imgs_to_drive:
  plt.savefig("images/mnist_perm_curve.pngs")

Puede verse que la red neuronal densa funciona exactamente igual en el caso de que los datos de entrada estén permutados o no. Sin embargo, en la red convolucional la precisión baja de manera dramática al estar perdiendo la información codificada en la estructura bidimensional de los datos.

In [ ]:
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten
from keras.models import Model

num_classes = 10
inputs = Input(shape=(28, 28, 1))
conv1_1 = Conv2D(32, kernel_size=(3, 3),
                 activation='relu', padding='same')(inputs)
conv1_2 = Conv2D(32, kernel_size=(3, 3),
                 activation='relu', padding='same')(conv1_1)
conv1_3 = Conv2D(32, kernel_size=(3, 3),
                 activation='relu', padding='same')(conv1_2)
maxpool1 = MaxPooling2D(pool_size=(2, 2))(conv1_3)
conv2_1 = Conv2D(32, (3, 3), activation='relu', padding='same')(maxpool1)
conv2_2 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv2_1)
conv2_3 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv2_2)
maxpool2 = MaxPooling2D(pool_size=(2, 2))(conv2_3)
flat = Flatten()(maxpool2)
dense = Dense(64, activation='relu')(flat)
predictions = Dense(num_classes, activation='softmax')(dense)

model = Model(inputs=inputs, outputs=predictions)

In [ ]:
model.summary()

Cambiamos la arquitectura de la red neuronal convolucional, en este caso vamos a ver que el entrenamiento de cada época va a tomar algo más de tiempo:

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
cnn_no_res = model.fit(X_train_images, y_train,
                       batch_size=128, epochs=10, verbose=1, validation_split=.1)